# Import libraries

In [ ]:
import os
import sys
print("Python version: {}". format(sys.version))
print("Python environment: {}".format(sys.executable))

import pandas as pd 
from pandas import ExcelWriter
from pandas import ExcelFile
#from openpyxl import load_workbook
print("pandas version: {}". format(pd.__version__))

import plotly_express as px
import matplotlib #collection of functions for scientific and publication-ready visualization
import matplotlib.pyplot as plt # for plotting
%matplotlib inline
print("matplotlib version: {}". format(matplotlib.__version__))
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()
print("seaborn version: {}". format(sns.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))
import scipy as sp #collection of functions for scientific computing and advance mathematics
from scipy import stats
from scipy.stats import norm, skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
from IPython.display import display
pd.options.display.max_columns = None
print("IPython version: {}". format(IPython.__version__)) 

import datetime
from datetime import datetime
from dateutil.parser import parse
from time import time

# to make this notebook's output identical at every run
np.random.seed(42)

In [ ]:
import sklearn
print("scikit-learn version: {}". format(sklearn.__version__))
# sklearn modules for preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# from imblearn.over_sampling import SMOTE  # SMOTE
# sklearn modules for ML model selection
from sklearn.model_selection import train_test_split  # import 'train_test_split'
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Libraries for data modelling
from sklearn import svm, tree, linear_model, neighbors
from sklearn import naive_bayes, ensemble, discriminant_analysis, gaussian_process
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor # import RandomForestRegressor
from sklearn.ensemble  import AdaBoostClassifier
from sklearn.ensemble  import GradientBoostingRegressor
from sklearn.linear_model import Lasso

# Common sklearn Model Helpers
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, RobustScaler
# from sklearn.datasets import make_classification

# sklearn modules for performance metrics
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
from sklearn.metrics import auc, roc_auc_score, roc_curve, recall_score, log_loss
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import average_precision_score
from sklearn.metrics import r2_score, make_scorer, mean_squared_error
print("scikit-learn libraries imported successfully")

# Other ML algorithms
from lightgbm import LGBMRegressor
print("lightgbm imported")
import xgboost as xgb
print("xgboost imported")
# from mlxtend.regressor import StackingCVRegressor, StackingRegressor
# print("StackingRegressor imported")

# Import data

In [ ]:
# importing the supplied dataset and storing it in a dataframe
training = pd.read_csv('train.csv')
# making copies of original datasets for rest of this kernel
df_train = training.copy()
print(df_train.shape)

In [ ]:
target = df_train['SalePrice']  #target variable
df_train = df_train.drop('SalePrice', axis=1) 

print("Training: {}, Target: {}, Test: {}".format(df_train.shape, target.shape, df_test.shape))

# Exploratory Data Analysis

## Quick EDA

In [ ]:
df_train_exp = df_train.copy() #make a copy of the training dataset for EDA purposes
print(df_train_exp.shape) 

In [ ]:
df_train_exp.head()

In [ ]:
# break down columns by data type
print("{} Numerical columns, {} Categorial columns".format(
    list(df_train_exp.select_dtypes(include=[np.number]).shape)[1],
    list(df_train_exp.select_dtypes(include = ['object']).shape)[1]))

In [ ]:
df_train_exp.columns.to_series().groupby(df_train_exp.dtypes).groups

In [ ]:
#list of columns with missing values
print("{} columns have missing values:".format(
    len(df_train_exp.columns[df_train_exp.isna().any()].tolist())))
df_train_exp.columns[df_train_exp.isna().any()].tolist()

In [ ]:
df_train_exp.describe() # let's have a look at variable types in our dataframe

In [ ]:
df_train_exp.hist(figsize=(18,18))
plt.show()

In [ ]:
# Testing for normal distribution hypothesis in numerical features
test_normality = lambda x: stats.shapiro(x.fillna(0))[1] < 0.01
numerical_features = [f for f in df_train_exp.columns if df_train_exp.dtypes[f] != 'object']
normal = pd.DataFrame(df_train_exp[numerical_features])
normal = normal.apply(test_normality)
print(not normal.any())

## Correlation Map

In [ ]:
# Calculate correlations
corr = training.corr(method='spearman')
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
# Heatmap
plt.figure(figsize=(15, 10))
sns.heatmap(corr,
            vmax=.5,
            mask=mask,
            #annot=True, 
            fmt='.2f',
            linewidths=.2, cmap="YlGnBu");

In [ ]:
# Find correlations with the target and sort
correlations = training.corr(method='spearman')['SalePrice'].sort_values(ascending=False)
correlations_abs = correlations.abs()
print('\nTop 10 correlations (absolute):\n', correlations_abs.head(11))


## Target Feature: SalePrice

In [ ]:
target_exp = target.copy() #make copy for exploratory purposes

In [ ]:
# let's see if there are any missing values (i.e. NA)
print("There are {} NA values in 'SalePrice'".format(target_exp.isnull().values.sum()))

In [ ]:
y = target_exp
plt.figure(1); plt.title('Log Normal')
sns.distplot(y, kde=False, fit=stats.lognorm)
plt.ylabel('Frequency')
print("Skewness: %f" % target_exp.skew())
# get mean and standard deviation
(mu, sigma) = norm.fit(target_exp)
print('Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma))

In [ ]:
# let's get some stats on the 'SalePrice' variable
print("Statistics for the supplied house prices training dataset:\n")
print("Minimum price: ${:,.2f}".format(np.min(target_exp)))
print("Maximum price: ${:,.2f}".format(np.max(target_exp)))
print("Mean price: ${:,.2f}".format(np.mean(target_exp)))
print("Median price ${:,.2f}".format(np.median(target_exp)))
print("Standard deviation of prices: ${:,.2f}".format(np.std(target_exp)))

In [ ]:
#  To get a visual of the outliers, let's plot a box plot.
sns.boxplot(y = target)
plt.ylabel('SalePrice (Log)')
plt.title('Price');

# count number of outliers after transformation is applied
Q1 = target.quantile(0.25)
Q3 = target.quantile(0.75)
IQR = Q3 - Q1
print("IQR value: {}\n# of outliers: {}".format(
    IQR,
    ((target < (Q1 - 1.5 * IQR)) | (target > (Q3 + 1.5 * IQR))).sum()))

# Data Preparation

## Log Transformation: Target Feature

In [ ]:
#applying log transformation to the Target Variable
target_tr = np.log1p(target)

# let's plot a histogram with the fitted parameters used by the function
sns.distplot(target_tr , fit=norm);
(mu, sigma) = norm.fit(target_tr)
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.title('Price (Log)');
print("Skewness: %f" % target_tr.skew())

In [ ]:
#  To get a visual of the outliers, let's plot a box plot.
sns.boxplot(y = target_tr)
plt.ylabel('SalePrice (Log)')
plt.title('Price');

# count number of outliers after transformation is applied
Q1 = target_tr.quantile(0.25)
Q3 = target_tr.quantile(0.75)
IQR = Q3 - Q1
print("IQR value: {}\n# of outliers: {}".format(
    IQR,
    ((target_tr < (Q1 - 1.5 * IQR)) | (target_tr > (Q3 + 1.5 * IQR))).sum()))

## Drop redundant columns

In [ ]:
df_train.drop(['Id'], axis=1, inplace=True)

## Convert data type

In [ ]:
numeric_features = list(df_train.select_dtypes(
        include=[np.number]).columns.values)
categ_features = list(df_train.select_dtypes(
    include=['object']).columns.values)

In [ ]:
numeric_features

In [ ]:
categ_features

In [ ]:
for col in numeric_features:
    df_train[col] = df_train[col].astype(float)

## Missing Values

In [ ]:
perc_na = (df_train.isnull().sum()/len(df_train))*100
ratio_na = perc_na.sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Values Ratio' :ratio_na})
print(missing_data.shape)
missing_data.head(20)

In [ ]:
# replacing NaNs in categorical features with "None"
df_train[categ_features] = df_train[categ_features].apply(lambda x: x.fillna("None"), axis=0)

In [ ]:
# imputing four numerical features with zero
for col in ("LotFrontage", 'GarageYrBlt', 'GarageArea', 'GarageCars'):
    df_train[col].fillna(0.0, inplace=True)

In [ ]:
# imputing other numerical features with median or mean
impute_method = "median"

if impute_method == "median": # replacing NaNs in numerical features with the median
    df_train[numeric_features] = df_train[numeric_features].apply(
        lambda x: x.fillna(x.median()), axis=0)
    print("Missing values imputed with median.")

elif impute_method == "mean": # replacing NaNs in numerical features with the mean
    df_train[numeric_features] = df_train[numeric_features].apply(
        lambda x: x.fillna(x.mean()), axis=0)
    print("Missing values imputed with mean.")

## Feature Engineering

In [ ]:
print("create combination of features.")
df_train['YrBltAndRemod']=df_train['YearBuilt']+df_train['YearRemodAdd']
df_train['TotalSF']=df_train['TotalBsmtSF'] + df_train['1stFlrSF'] + df_train['2ndFlrSF']

df_train['Total_sqr_footage'] = (df_train['BsmtFinSF1'] + df_train['BsmtFinSF2'] +
                                 df_train['1stFlrSF'] + df_train['2ndFlrSF'])

df_train['Total_Bathrooms'] = (df_train['FullBath'] + (0.5 * df_train['HalfBath']) +
                               df_train['BsmtFullBath'] + (0.5 * df_train['BsmtHalfBath']))

df_train['Total_porch_sf'] = (df_train['OpenPorchSF'] + df_train['3SsnPorch'] +
                              df_train['EnclosedPorch'] + df_train['ScreenPorch'] + 
                             df_train['WoodDeckSF'])

In [ ]:
print("create boolean features.")
df_train['haspool'] = df_train['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
df_train['has2ndfloor'] = df_train['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
df_train['hasgarage'] = df_train['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
df_train['hasbsmt'] = df_train['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
df_train['hasfireplace'] = df_train['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
print("Transformed numerical features that should be considered as strings.")
df_train['MSSubClass'] = df_train['MSSubClass'].apply(str)
df_train['YrSold'] = df_train['YrSold'].astype(str)
df_train['MoSold'] = df_train['MoSold'].astype(str)
df_train['YrBltAndRemod'] = df_train['YrBltAndRemod'].astype(str)

In [ ]:
numeric_features = list(df_train.select_dtypes(include=[np.number]).columns.values)
categ_features = list(df_train.select_dtypes(include=['object']).columns.values)

In [ ]:
numeric_features

In [ ]:
categ_features

In [ ]:
# Transform numerical columns with skewness factor > 0.5
# This is optional
print("Transformed numerical columns with high skewness factor.")
skew_features = df_train[numeric_features].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index
for i in skew_index:
    df_train[i] = boxcox1p(df_train[i], boxcox_normmax(df_train[i]+1))

In [ ]:
# Label Encoding
df_train = pd.get_dummies(df_train)

In [ ]:
df_train.shape

In [ ]:
# scale features
feature_scaling = "RobustScaler"

if feature_scaling == 'MinMaxScaler':
    scaler = MinMaxScaler(feature_range=(0, 1))
    for col in numeric_features:
        df_train[[col]] = scaler.fit_transform(df_train[[col]])
    print("Performed feature Scaling with MinMaxScaler.")

elif feature_scaling == 'StandardScaler':
    scaler = StandardScaler()
    for col in numeric_features:
        df_train[[col]] = scaler.fit_transform(df_train[[col]])
    print("Performed feature Scaling with StandardScaler.")

elif feature_scaling == "RobustScaler":
    scaler = RobustScaler()
    for col in numeric_features:
        df_train[[col]] = scaler.fit_transform(df_train[[col]])
    print("Performed feature Scaling with RobustScaler.")

## Final df validation

In [ ]:
# let's check that we no longer have any missing values
perc_na = (df_train.isnull().sum()/len(df_train))*100
ratio_na = perc_na.sort_values(ascending=False)
missing_data = pd.DataFrame({'missing_ratio' :ratio_na})
missing_data = missing_data.drop(missing_data[missing_data.missing_ratio == 0].index)
missing_data.head(5)

# Machine Learning Models

## training and testing data split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train,
                                                    target_tr,
                                                    test_size=0.2,
                                                    random_state=42)

In [ ]:
print("Training Data Shape: {}".format(df_train.shape))
print("X_train Shape: {}".format(X_train.shape))
print("X_test Shape: {}".format(X_test.shape))

## Baseline results

In [ ]:
models = []
models.append(('Ridge Regression', Ridge(alpha=1.0)))
models.append(('ElasticNet', ElasticNet()))
models.append(('Random Forest', RandomForestRegressor(
    n_estimators=100, random_state=7)))
models.append(('Lasso', Lasso(random_state=42)))
models.append(('XGBoost Regressor', xgb.XGBRegressor(objective='reg:squarederror', 
                                                     random_state=42)))
models.append(('Gradient Boosting Regressor', GradientBoostingRegressor()))
models.append(('LGBM Regressor',LGBMRegressor(objective='regression')))
models.append(('SVR',SVR()))

# set table to table to populate with performance results
rmse_results = []
names = []
col = ['Algorithm', 'RMSE Mean', 'RMSE SD']
df_results = pd.DataFrame(columns=col)

# evaluate each model using cross-validation
kfold = model_selection.KFold(n_splits=5, shuffle = True, random_state=7)
i = 0
for name, model in models:
    # -mse scoring
    cv_mse_results = model_selection.cross_val_score(
        model, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
    # calculate and append rmse results
    cv_rmse_results = np.sqrt(-cv_mse_results)
    rmse_results.append(cv_rmse_results)
    names.append(name)
    df_results.loc[i] = [name,
                         round(cv_rmse_results.mean(), 4),
                         round(cv_rmse_results.std(), 4)]
    i += 1
df_results.sort_values(by=['RMSE Mean'], ascending=True).reset_index(drop=True)

In [ ]:
fig = plt.figure(figsize=(15, 8))
fig.suptitle('Algorithm RMSE Comparison')
ax = fig.add_subplot(111)
plt.boxplot(rmse_results)
ax.set_xticklabels(names)
plt.show();

## Fine-Tuning ML Hyper-Parameters

### XGBoost

In [ ]:
xgb_regressor = xgb.XGBRegressor(random_state=42)

In [ ]:
parameters_xgb = {'n_estimators':range(10, 200, 10), 
             'learning_rate':[0.05,0.060,0.070], 
             'max_depth':[3,5,7],
             'min_child_weight':[1,1.5,2]}
grid_obj_xgb = RandomizedSearchCV(xgb_regressor, 
                                 parameters_xgb,
                                 scoring = 'r2', 
                                 cv = 5,
                                 n_jobs = -1,
                                 n_iter = 100,
                                 random_state= 99)
grid_fit_xgb = grid_obj_xgb.fit(X_train, y_train)
xgb_opt = grid_fit_xgb.best_estimator_

print("best params: " + str(grid_fit_xgb.best_params_))
print('best score:', grid_fit_xgb.best_score_)

In [ ]:
# r2 on testing data
r2_score(y_test, xgb_opt.predict(X_test)) 

### Random Forest Regressor

In [ ]:
rf_regressor = RandomForestRegressor(random_state=42)

parameters = {'n_estimators':range(10, 200, 10), 
              'min_samples_leaf':range(5, 40, 5), 
              'max_depth':range(3, 5, 1)}
grid_obj_rf = RandomizedSearchCV(rf_regressor, 
                                 parameters,
                                 scoring = 'r2', 
                                 cv = 5,
                                 n_jobs = -1,
                                 n_iter = 100,
                                 random_state= 99)
grid_fit_rf = grid_obj_rf.fit(X_train, y_train)
rf_opt = grid_fit_rf.best_estimator_

print("best params: " + str(grid_fit_rf.best_params_))
print('best score:', grid_fit_rf.best_score_)

In [ ]:
# r2 on testing data
r2_score(y_test, rf_opt.predict(X_test)) 

## Feature importance

### XGBoost

In [ ]:
best_parameters_xgb = {'n_estimators': 190, 'min_child_weight': 2, 'max_depth': 3, 'learning_rate': 0.07}
xgb_reg = xgb.XGBRegressor(**best_parameters_xgb)

In [ ]:
xgb_model = xgb_reg.fit(df_train, target_tr)

In [ ]:
xgb.plot_importance(xgb_model,  max_num_features=20 , importance_type='gain')

### Random Forest Regressor

In [ ]:
best_parameters_rf = {'n_estimators': 110, 'min_samples_leaf': 5, 'max_depth': 4}

In [ ]:
rf_regressor = RandomForestRegressor(**best_parameters_rf)

In [ ]:
rf_model = rf_regressor.fit(df_train, target_tr)

In [ ]:
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1] 
names = [df_train.columns[i] for i in indices] 
plt.figure(figsize=(15, 7)) 
plt.title("Top 10 Most Important Features") 
plt.bar(range(10), importances[indices][:10]) 
plt.xticks(range(10), names[:10], rotation=90) 
plt.show() 